# pseudo code 

In [ ]:
다른 버스들과 마찬가지로 1112번도 사색에서 도착정보(사색의 광장에 버스가 도착하기 전까지 몇분 남았는지)를 수집할 수 없다. 
그래서 생대역의 도착 정보를 이용할 것이다.
생대역에서 도착정보 없음 계속 뜨다가 처음 정상처리이 뜨는 순간이 사색에 버스가 도착했다는 뜻이다. 그때마다 시간을 기록할 것이다. 
그리고 사색의 광장에 버스가 도착했을 당시에, 운행중인 1112번 버스들의 위치(정류장)도 함께 수집해서 규칙성이 존재하는지 볼 것이다. 
이 코드의 특징:
1112번외에도 다른 버스들도 똑같이 수집 가능하고,
불연속 점을 도출하기 위해 초마다 api 호출을 안해도 되고, 
[사색 도착시간,운행중인 ㅇㅇ버스들의 현 위치]를 수집할 수 있다.

past=없다
1분마다 요청을 한다.
    result=도착정보가 있다 or 없다
    if past!=result:
        if result==도착정보 있다
            처음으로 '있다'가 나왔다. '현시간=사색 도착 시간'을 저장한다. 
    past=result 값 임시 저장

# 실제코드

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

bus_time=[]
p5100 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
             'routeId':'200000115','stationId':'228000704'}#5100 생과대
p1112 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
         'routeId':'234000016','stationId':'228000704'}#1112 생과대

def repet(params): 
    url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
    response = requests.get(url, params=params);
    soup=BeautifulSoup(response.text,'xml')
    return soup

def current_buses(): # 사색의 광장에 버스가 도착했을 때, 운행중인 같은 노선의 버스들의 위치(정류장)를 수집  
    url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
    params ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
              'routeId':'200000115'}#5100번 노선id
    response=requests.get(url,params=params)
    soup=BeautifulSoup(response.text,'xml')
    locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
    return locations


In [11]:
past=4
for x in range(170): # 10분간
    soup=repet(p5100)
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        bus_time.append([soup.queryTime.text,current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",bus_time)
        time.sleep(60)
    except Exception as e:
        continue
df=pd.DataFrame(bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])
df.to_csv('5100_fri.csv',index=False)


0 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0]]
1 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0]]
2 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0]]
3 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0]]
4 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], [

15 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 4

23 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 4

28 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 4

32 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 4

...

165 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 

166 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 

167 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 

168 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 

169 번째 시도:  [['2022-07-08 13:10:01.985', [3, 9, 13, 24, 26, 32, 37, 39, 42, 49], 0], ['2022-07-08 13:11:02.694', [4, 10, 14, 24, 26, 33, 37, 39, 43, 49], 0], ['2022-07-08 13:12:03.700', [5, 11, 14, 24, 27, 33, 38, 40, 43, 50], 0], ['2022-07-08 13:13:03.956', [5, 11, 14, 24, 27, 34, 38, 40, 43, 52], 0], ['2022-07-08 13:14:04.274', [7, 12, 15, 24, 27, 35, 38, 40, 43, 53], 0], ['2022-07-08 13:15:04.749', [8, 13, 15, 24, 27, 35, 39, 40, 43, 54], 0], ['2022-07-08 13:16:05.817', [8, 13, 16, 25, 28, 36, 39, 41, 43], 0], ['2022-07-08 13:17:06.042', [9, 14, 16, 25, 28, 37, 39, 41, 43], 0], ['2022-07-08 13:18:06.282', [9, 14, 16, 26, 28, 37, 40, 42, 43], 0], ['2022-07-08 13:19:07.569', [9, 14, 16, 26, 29, 37, 40, 42, 47], 0], ['2022-07-08 13:20:07.794', [10, 15, 17, 26, 29, 38, 40, 43, 49], 0], ['2022-07-08 13:21:07.987', [11, 15, 17, 27, 29, 38, 40, 44, 49], 0], ['2022-07-08 13:22:08.290', [11, 15, 18, 27, 29, 38, 41, 45, 49], 0], ['2022-07-08 13:23:09.093', [1, 12, 16, 18, 27, 30, 38, 41, 46, 

# [단계2]feature engineering

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer

df=pd.read_csv('5100_thu.csv')
mb = sklearn.preprocessing.MultiLabelBinarizer()
if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
    df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
lsts=pd.DataFrame(entity,columns=mb.classes_)
lsts.columns=[f'station{x}' for x in lsts.columns]

df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)
df['시간'] = df['시간'].apply(pd.to_datetime) # string 값인 시간을 datetime객체로 바꾼다. 
df['요일'] = df['시간'].dt.weekday #월요일 0 화요일 1
df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
df = df.set_index('시간')
df

,arrived,station1,station2,station3,station4,station5,station6,station7,station8,station9,...,station46,station47,station48,station49,station50,station51,station52,station53,station54,요일
시간,,,,,,,,,,,,,,,,,,,,,
2022-07-07 13:50:35.436,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 13:51:35.623,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 13:52:35.778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 13:53:35.981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 13:54:36.168,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-07 15:52:39.346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 15:53:39.548,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2022-07-07 15:54:39.767,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [112]:
df=pd.DataFrame(bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])
df.to_csv('5100_wed8-9.csv',index=False)